> The models proposed recently for neural machine translation often belong to a family of encoder–decoders and encode a source sentence into a fixed-length vector from which a decoder generates a translation. In this paper, we conjecture that the use of a fixed-length vector is a bottleneck in improving the performance of this basic encoder–decoder architecture, and propose to extend this by allowing a model to automatically (soft-)search for parts of a source sentence that are relevant to predicting a target word, without having to form these parts as a hard segment explicitly.

Instead of a fixed vector outputted from the source sequence, we can insert source context at each decoder output step.

Instead, for each output we generate, we soft search for source words that best align with the output. Then predicts the target word that has highest probability given the context vectors.
This gives more robust context and linking from any source to an output.

# 2 Background

Maximizing the conditional probability p(y|x), where y is the target sentence
